In [1]:
!pip install --quiet streamlit streamlit-folium folium geopy pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.2 MB/s eta 0:00:00


In [5]:
%%writefile app.py
import streamlit as st
import requests, folium, pytz
from datetime import datetime
from streamlit_folium import st_folium
from geopy.geocoders import Nominatim

# ---------- CONFIG ----------
PRIMARY_ISS_URL  = "https://api.wheretheiss.at/v1/satellites/25544"   # HTTPS ✅
FALLBACK_ISS_URL = "http://api.open-notify.org/iss-now.json"          # HTTP (only if needed)
ISS_PASSES_URL   = "http://api.open-notify.org/iss-pass.json"         # may be offline
ASTROS_URL       = "http://api.open-notify.org/astros.json"

# ---------- HELPERS ----------
def _parse_wtia(j):
    return j["latitude"], j["longitude"], j["timestamp"]

def _parse_open_notify(j):
    return float(j["iss_position"]["latitude"]), float(j["iss_position"]["longitude"]), j["timestamp"]

@st.cache_data(ttl=15, show_spinner=False)
def get_iss_location():
    """
    Try the HTTPS WTIA API first; fall back to Open‑Notify if Colab blocks HTTP.
    """
    for url, parser in [(PRIMARY_ISS_URL, _parse_wtia), (FALLBACK_ISS_URL, _parse_open_notify)]:
        try:
            r = requests.get(url, timeout=10)
            r.raise_for_status()
            return parser(r.json())
        except Exception:
            continue
    st.error("Couldn’t reach any ISS location API right now. Please try again later.")
    st.stop()

@st.cache_data(ttl=60*60)
def get_astronauts():
    return requests.get(ASTROS_URL, timeout=10).json()["people"]

@st.cache_data(ttl=60)
def get_pass_times(lat, lon, n=5):
    params = {"lat": lat, "lon": lon, "n": n}
    r = requests.get(ISS_PASSES_URL, params=params, timeout=10)
    r.raise_for_status()
    return r.json()["response"]

def fmt_time(ts, tz="UTC"):
    return datetime.fromtimestamp(ts, pytz.timezone(tz)).strftime("%Y-%m-%d %H:%M:%S")

# ---------- STREAMLIT ----------
st.set_page_config("ISS Tracker", "🛰️", layout="wide")
st.title("🛰️ International Space Station — Live Tracker")

tab_live, tab_pass, tab_people = st.tabs(["🌍 Live Map", "🔭 Next Pass", "👩‍🚀 Astronauts Aboard"])

# -------- LIVE MAP --------
with tab_live:
    st.markdown("### Current position (auto‑refreshes every 15 s)")
    lat, lon, ts = get_iss_location()
    m = folium.Map(location=[lat, lon], zoom_start=2, control_scale=True)
    folium.Marker(
        [lat, lon],
        tooltip=f"ISS @ {fmt_time(ts)} UTC",
        icon=folium.Icon(color="red", icon="rocket", prefix="fa")
    ).add_to(m)
    st_folium(m, width=700, height=500)

# -------- NEXT PASS --------
with tab_pass:
    st.markdown("### When will the ISS fly over you?")
    addr = st.text_input("Enter a city / address:")
    if st.button("Find next passes"):
        if not addr:
            st.warning("Please enter a location first 😊")
        else:
            geolocator = Nominatim(user_agent="iss_tracker")
            loc = geolocator.geocode(addr)
            if loc is None:
                st.error("Location not found. Try something more specific.")
            else:
                try:
                    passes = get_pass_times(loc.latitude, loc.longitude)
                    st.success(f"Next {len(passes)} passes (UTC):")
                    for p in passes:
                        st.write(f"- **{fmt_time(p['risetime'])}** — duration {p['duration']} s")
                except Exception:
                    st.info("Sorry, pass‑prediction data isn’t available right now.")

# -------- ASTRONAUTS --------
with tab_people:
    st.markdown("### Who’s in space right now?")
    people = get_astronauts()
    st.write(f"Total astronauts in space: **{len(people)}**")
    for p in people:
        st.markdown(f"- **{p['name']}** aboard *{p['craft']}*")

st.button("🔄 Manual refresh")

Overwriting app.py


In [6]:
!ngrok config add-authtoken *Your_Auth_Token*

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
from pyngrok import ngrok
public_url = ngrok.connect(8501, "http").public_url
print(f"🌐 Open your app at: {public_url}")
!streamlit run app.py &>/dev/null &

🌐 Open your app at: https://9808-34-73-204-29.ngrok-free.app
